# Second Data Cleaning 

In [2]:
import pandas as pd
import re
from fuzzywuzzy import fuzz

C:\Users\broba\AppData\Local\Programs\Python\Python313\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
df1 = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Cleaning Data\df1_unique_streets_only.csv")  
df2 = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Cleaning Data\df2_unique_streets_zipcode.csv")  

In [4]:
replacements = {
    "ROAD": "RD", "RD": "RD",
    "STREET": "ST", "ST": "ST",
    "AVENUE": "AVE", "AVE": "AVE",
    "DRIVE": "DR", "DR": "DR",
    "COURT": "CT", "CT": "CT",
    "LANE": "LN", "LN": "LN",
    "CIRCLE": "CIR", "CIR": "CIR",
    "TRAIL": "TRL", "TRL": "TRL",
    "PARKWAY": "PKWY", "PKWY": "PKWY",
    "HIGHWAY": "HWY", "HWY": "HWY",
    "PLACE": "PL", "PL": "PL",
    "PATH": "PATH",
    "WAY": "WAY",
    "LOOP": "LOOP",
    "COVE": "CV", "CV": "CV",
    "NORTH": "N", "N": "N",
    "SOUTH": "S", "S": "S",
    "EAST": "E", "E": "E",
    "WEST": "W", "W": "W",
    "NORTHEAST": "NE", "NE": "NE",
    "NORTHWEST": "NW", "NW": "NW",
    "SOUTHEAST": "SE", "SE": "SE",
    "SOUTHWEST": "SW", "SW": "SW",
    "NB": "NB", "SB": "SB", "EB": "EB", "WB": "WB"
}

direction_garbage = {"BLOCK", "NB", "SB", "EB", "WB", "NORTHBOUND", "SOUTHBOUND", "EASTBOUND", "WESTBOUND"}

ordinals = {
    "FIRST": "1ST", "SECOND": "2ND", "THIRD": "3RD", "FOURTH": "4TH",
    "FIFTH": "5TH", "SIXTH": "6TH", "SEVENTH": "7TH", "EIGHTH": "8TH",
    "NINTH": "9TH", "TENTH": "10TH"
}

def advanced_clean(s):
    if pd.isna(s) or s == 'nan':
        return ''
    s = s.upper()
    s = re.sub(r"^\d+\s+", "", s)  # deleting the first numbers 
    s = re.sub(r"[^\w\s]", "", s)  # delete punctuation
    s = re.sub(r"\d+ BLOCK", "", s)  #  delete BLOCK  
    s = re.sub(r"BLOCK", "", s)  # delete BLOCK
    s = re.sub(r"\d+", "", s)  # deleting the numbers
    s = re.sub(r"1/2|1 2", "", s)  # delete 1/2
    s = re.sub(r"SERVICE RD|SVRD|FWY|EXPY|EXPRESSWAY|IH|US|SH|FM", "", s)  # delete road expressions
    s = re.sub(r"\s+", " ", s)  # delete extra spaces
    tokens = s.split()
    clean_tokens = []
    for t in tokens:
        if t in direction_garbage:
            continue
        t = ordinals.get(t, t)
        t = replacements.get(t, t)
        clean_tokens.append(t)
    return " ".join(clean_tokens).strip()

## Second Match Rate

In [5]:
df1['street_clean1'] = df1['street_clean1'].apply(advanced_clean)
df1_unique = df1.drop_duplicates(subset=["street_clean1"])

In [6]:
df2['street_clean2'] = df2['street_clean2'].apply(advanced_clean)
df2_unique = df2.drop_duplicates(subset=["street_clean2", "POSTCODE"])

In [7]:
df1_unique.to_csv('df1_cleaned_final.csv', index=False)
df2_unique.to_csv('df2_cleaned_final.csv', index=False)

In [8]:
common = set(df1_unique["street_clean1"]) & set(df2_unique["street_clean2"])
common.discard('')
match_rate = (len(common) / len(df1_unique["street_clean1"])) * 100 if len(df1_unique) > 0 else 0

In [11]:
print("Match rate :", match_rate, "%")
print("number of common streets:", len(common))
print("common examples:", list(common)[:10])

Match rate : 40.43927648578811 %
number of common streets: 1878
common examples: ['MALONE CT', 'PETES PATH', 'TALLOW CT', 'SINGLETREE AVE', 'NEWFOUNDLAND CIR', 'FRONTIER TRL', 'BROOMFLOWER DR', 'PARKINSON DR', 'CONNIE ST', 'FORD ST']


## Second Integrate

In [15]:
crimes = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Datasets\Municipal_Court_Caseload_Information_FY_2023.csv")
df2 = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Cleaning Data\2\df2_cleaned_final.csv")  # cleaned Street-ZIP
neigh = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Datasets\austin_zip_to_neighborhood_scraped.csv")
pop = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Datasets\austin_population_by_zip_scraped.csv")

In [16]:
crimes['street_clean'] = crimes['Offense Street Name'].apply(advanced_clean)

In [17]:
crimes_with_zip = pd.merge(crimes, df2, left_on='street_clean', right_on='street_clean2', how='left')

In [18]:
crimes_with_neigh = pd.merge(crimes_with_zip, neigh, left_on='POSTCODE', right_on='ZIP_Code', how='left')

In [19]:
valid_pop = pop[pop['Population'] > 0]
crimes_integrated = pd.merge(crimes_with_neigh, valid_pop[['ZIP_Code', 'Population']], left_on='POSTCODE', right_on='ZIP_Code', how='left')

In [20]:
analysis = crimes_integrated.groupby('Neighborhood').agg({
    'Offense Case Type': 'count',
    'Population': 'mean'
}).reset_index()
analysis.rename(columns={'Offense Case Type': 'Crime_Count'}, inplace=True)
analysis['Crime_Rate_per_1000'] = (analysis['Crime_Count'] / analysis['Population']) * 1000
analysis = analysis.sort_values('Crime_Rate_per_1000', ascending=False)

In [21]:
crimes_integrated.to_csv('final_integrated_all.csv', index=False)
analysis.to_csv('analysis_neighborhood_rate.csv', index=False)

In [22]:
print("Match rate:", (crimes_integrated['POSTCODE'].notna().mean() * 100))
print(analysis.head(10))

Match rate: 94.64126805347841
                                         Neighborhood  Crime_Count  \
6                                     Downtown Austin       106361   
3                                          Cherrywood        32437   
11                                          Hyde Park        62293   
23                                           Rosedale        30070   
8                                         East Austin        68017   
17                        North University, Hyde Park        95018   
29                             Tarrytown, Clarksville        37075   
18                                    Northwest Hills        28742   
14                              Mueller, Windsor Park        36221   
26  South Lamar, Zilker, Bouldin Creek, Travis Hei...        37575   

      Population  Crime_Rate_per_1000  
6   11625.000000          9149.333333  
3    6618.000000          4901.329707  
11  17071.000000          3649.053951  
23   8426.000000          3568.715879  

## Persisting the Integrated Data in a DBMS

In [23]:
import sqlite3

In [24]:
crimes_integrated = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Integration\final_integrated_all.csv")

C:\Users\broba\AppData\Local\Temp\ipykernel_9432\178137058.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  crimes_integrated = pd.read_csv(r"E:\Bahare (important)\uni\Master\Data Managenement\Integration\final_integrated_all.csv")


In [25]:
conn = sqlite3.connect('austin_crimes_project.db')
crimes_integrated.to_sql('crimes_data', conn, if_exists='replace', index=False)

752902